In [11]:
import race_results_scanner
import header_scanner as hs
import google_cloud_downloader as gcd
from PyPDF2 import PdfReader
import pandas as pd
import os

pdf = "Error_PDFS/AQU_11_04_2022.pdf"


In [ ]:
#Testing for whole pdf
master_df = pd.DataFrame()
page_list = race_results_scanner.get_page_list(pdf)
print(page_list)
    
reader = PdfReader(pdf) #File to be scanned
number_of_pages = len(reader.pages) #Number of pages
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()
i = 0
for page in page_list:
    header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
    result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
                
    #Combine into page DF
    top_table = result_tables[0]
    bottom_table = result_tables[1]
    #Dropping horse name for merge
    bottom_table = bottom_table.drop("Horse Name", axis = 1)
    merged_df = top_table.join(bottom_table.set_index("Pgm"), on = "Pgm", rsuffix = "_RLP")
    for field,value in header.items():
        merged_df[field] = value
        #Changing all cols to object type 
        merged_df = merged_df.astype(object)
        #For every page df
        #Merge into master pdf df
        if(len(pdf_df) < 1):
            pdf_df = merged_df
        else:
            pdf_df = pd.merge(pdf_df, merged_df, how = 'outer')
            
        print(pdf_df)
        #Merging into master 
        if(len(master_df) < 1):
            master_df = pdf_df
        else:
            master_df = pd.merge(master_df,pdf_df, how = 'outer')

The PDF <_io.BufferedReader name='Error_PDFS/AQU_11_04_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


[{'page_num': 0, 'horse_count': 7}, {'page_num': 1, 'horse_count': 6}, {'page_num': 2, 'horse_count': 7}, {'page_num': 3, 'horse_count': 7}, {'page_num': 4, 'horse_count': 9}, {'page_num': 6, 'horse_count': 8}, {'page_num': 7, 'horse_count': 11}, {'page_num': 9, 'horse_count': 7}, {'page_num': 10, 'horse_count': 5}, {'page_num': 11, 'horse_count': 9}]
   Page                                   Text      x_1      y_1      x_2  \
0     0                             Last Raced    9.920  593.170   55.947   
2     0                       Fractional Times    9.920  557.870  161.331   
4     0  Past Performance Running Line Preview  209.385  437.045  363.209   
5     0                                    Fin  406.767  358.401  427.339   
6     0                               Trainers    9.920  337.767  535.557   

       y_2  
0  668.314  
2  583.870  
4  445.045  
5  433.545  
6  345.767  
   Page  Table      Top    Left   Bottom   Right
0   0.0    1.0  135.830    7.92  207.130  559.44
1   0.0

In [ ]:
master_df

In [13]:
#Testing for page
page_list = race_results_scanner.get_page_list(pdf)
target_page = 11

page_list_index = 0
for page in page_list:
    if(target_page == page['page_num']):
        break
    page_list_index += 1
    
reader = PdfReader(pdf) #File to be scanned
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()

page = page_list[page_list_index]
header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
for field,value in header.items():
    print(field ,':', value)
result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
print(result_tables[0])
print(result_tables[1])

The PDF <_io.BufferedReader name='Error_PDFS/AQU_11_04_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


track :  AQUEDUC /T
date :  November 4, 2022
race_num :  10
criteria :  FOR MAIDENS, FILLIES TWO YEARS OLD NY-BRED WEIGHT, 119 LBS.(NON-STARTERS
track_length :  FOR A CLAIMING PRICE OF $40,000 OR LESS IN THE LAST 3 STARTS PREFERRED). (Rail at 32 feet). (S) About One And One Sixteenth Miles On The Turf
purse :  $70,000
weather :  Cloudy
track_type :  Firm
off_time :  5 16
start :  Good for all
track : Aqueduct
date : November 4, 2022
race_num : 10
criteria : FOR MAIDENS, FILLIES TWO YEARS OLD NY-BRED WEIGHT, 119 LBS(NON-STARTERS
track_length :  One And One Sixteenth Miles On The Turf
purse : $70,000
weather : Cloudy
track_type : Firm
off_time : 5:16
start : Good for all
race_type : maiden special weight
Looking for tables...
bound_num =  10
bound_num =  -10
bound_num =  20
bound_num =  -20
bound_num =  30
bound_num =  -30
bound_num =  40
bound_num =  -40
bound_num =  50
bound_num =  -50
Found table top bound at 118.68599999999992
bound_num =  10
Found table bottom bound at 225.713999999